In [37]:
import pandas as pd
import csv

In [38]:
def import_data(file_name):
    data = []
    with open(file_name, 'r') as file:
        reader = csv.reader(file, delimiter=' ')
        for row in reader:
            data.append([float(val) for val in row])
    return data

In [39]:
def k_means_clustering(data, attribute_index, k):
    from sklearn.cluster import KMeans
    import numpy as np
    
    labels = ['Low', 'Medium', 'High']
    # Prepare data for clustering
    X = np.array([row[attribute_index] for row in data]).reshape(-1, 1)

    # Perform K-means clustering
    try:
        kmeans = KMeans(n_clusters=k, random_state=0,n_init='auto').fit(X)
        centroids = kmeans.cluster_centers_.flatten().tolist()
        cluster_assignments = kmeans.labels_.tolist()
        cluster_assignments = [labels[i] for i in cluster_assignments]
        return cluster_assignments, centroids
    except:
        return X.T.tolist()[0], 0

In [40]:

def conflict_handling(df):
    handle = df.values.tolist()
    indices_to_drop = set()

    for i in range(len(handle)):
        for j in range(i + 1, len(handle)):
            if handle[i][:-1] == handle[j][:-1]: 
                if handle[i][-1] != handle[j][-1]:
                    indices_to_drop.add(i)
                    indices_to_drop.add(j)

    df = df.drop(index=list(indices_to_drop))
    return df


In [41]:
df1 = pd.read_csv("/kaggle/input/result/Result.csv")
# df2 = pd.read_csv("/kaggle/input/dataa12/diabetes_prediction_dataset.csv")
cols1 = df1.columns.to_list()
# cols2 = df2.columns.to_list()

In [42]:
def FIS(df):
    # file_name = "heart.txt"
    # file_name1 = "heart_Label.txt"
    y = df.iloc[:,-1:]
    data = df.values.tolist()
    data_label = [0,1]

    # Số cụm tối đa
    k = 3

    # Phân cụm K-means cho mỗi thuộc tính
    clusters = []
    centroids = []

    for i in range(len(data[0])-1):
        cluster_assignments, centroid = k_means_clustering(data, i, k)
        clusters.append(cluster_assignments)
        centroids.append(centroid)

    # list_labels = [point for point in data_label]
    # clusters.append(list_labels)

    # In ra thông tin các cụm và tâm cụm
    # for i in range(len(clusters)):
    #     print(f"Thuộc tính {i + 1}: {clusters[i]}")
    # for i in range(len(centroids)):
    #     print(f"Tâm cụm {i + 1}: {centroids[i]}")

    out_df = pd.DataFrame(clusters).T
    out_df = pd.concat([out_df,y],axis=1)
    out_df.columns = df.columns
    return out_df

In [43]:
f_df1 = FIS(df1)
f_df1

,Contrast Feature,Dissimilarity Feature,Homogeneity Feature,Energy Feature,Correlation Feature,ASM Feature,diagnostic
0,Low,Low,Low,Medium,Low,High,5
1,Low,Low,Low,High,Low,Medium,1
2,Low,High,High,Low,Low,Low,3
3,Low,Low,Low,High,Low,Low,3
4,Low,Low,Low,High,Low,Medium,1
...,...,...,...,...,...,...,...
2293,Low,Low,Low,Low,Low,Low,3
2294,Low,Low,Low,High,Low,Medium,1
2295,Low,High,High,Low,Medium,Low,4
2296,Low,Low,Low,High,Low,Medium,1


In [44]:
# f_df2 = FIS(df2)
# print(f_df2)


In [45]:
def merge(df1, df2):
    df2.columns = df1.columns
    return pd.concat([df1,df2],axis=0, ignore_index=True)

In [46]:
# df = merge(f_df1,f_df2)

In [47]:
df = f_df1

In [48]:
df = conflict_handling(df)

In [49]:
# df = df.reset_index().drop(columns=['index'])
# df

In [50]:
df.to_csv("Data.csv", index=False)

In [51]:
from sklearn.model_selection import train_test_split

y = df.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.33, random_state=42)

In [52]:
X_train.to_csv('train.csv', index=False)

In [53]:
X_test.to_csv('test.csv', index=False)

In [54]:
# print(df['Diagnosis'].value_counts()[0])
# print(df['Diagnosis'].value_counts()[1])